In [1]:
import os
os.environ["WANDB_DISABLED"]="true"

In [2]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

In [3]:
import transformers
print(transformers.__version__)

In [4]:
model_checkpoint = "Helsinki-NLP/opus-mt-mul-en"

In [5]:
from datasets import load_dataset, load_metric
metric = load_metric("sacrebleu")

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [7]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

In [9]:
import pandas as pd

In [10]:
train_df = pd.read_csv('../input/ro-fa-clean-translate/train_df.csv')
test_df = pd.read_csv('../input/ro-fa-clean-translate/test_df.csv')
train_df['ro'] = train_df.ro.astype(str).apply(lambda x: x.replace('</i>', ''))
test_df['ro'] = test_df.ro.astype(str).apply(lambda x: x.replace('</i>', ''))
train_df['fa'] = train_df.fa.astype(str).apply(lambda x: x.replace('</i>', ''))
test_df['fa'] = test_df.fa.astype(str).apply(lambda x: x.replace('</i>', ''))

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train_df, shuffle=True, test_size=len(train_df)//20, random_state=42)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

import datasets
from datasets import Dataset, dataset_dict

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

raw_datasets = dataset_dict.DatasetDict({'train':train_dataset, 'validation':val_dataset, 'test':test_dataset})

In [11]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "fa"
target_lang = "ro"
def preprocess_function(dataset):
    inputs = dataset[source_lang]
    targets = dataset[target_lang]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
preprocess_function(train_dataset[:2])

In [13]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [14]:
tokenized_datasets

In [15]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [16]:
batch_size = 64
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True    
)

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [18]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
import os
for dirname, _, filenames in os.walk('opus-mt-en-ro-finetuned-en-to-ro'):
    for filename in filenames:
        print(os.path.join(dirname, filename))